# Predict name based on first several letters

In [ ]:
import torch
import torch.nn.functional as F
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(f"{torch.cuda.is_available()=}")
device="cpu"
if torch.cuda.is_available():
    device="cuda:0"
print(device)

In [ ]:
import string
itos = {0:'.'}
for i, c in enumerate(string.ascii_lowercase):
    itos[i+1]=c  
stoi = {s:i for i, s in itos.items()}
print(stoi)
voc_size=len(itos)
print(f"{voc_size=}")

In [ ]:
def encode(ss):
    res = [stoi[c] for c in ss]
    return res

def decode(ii, tilldot=False):
    ch = False
    res = []
    for i in ii:
        if i == 0 and tilldot and ch:
            break
        if not(ch) and i != 0:
            ch = True
        res.append(itos[i])
    return ''.join(res)

In [ ]:
names_f = "names.txt"
with open(names_f) as f:
    words = f.read().splitlines()

#random.seed(42)
random.shuffle(words)
print(words[:3])
print(len(words))

In [ ]:
def add_word(w, bsz, X, Y):
    x = "."*bsz
    xi = [0]*bsz
    for y in w:
        yi = stoi[y]
        X.append(xi)
        Y.append(yi)
        xi = xi[1:]
        xi.append(yi)
    X.append(xi)
    Y.append(0)

In [ ]:
att=3
emb=2
hidden = 100

Xa, Ya = [], []
for w in words:
    add_word(w, att, Xa, Ya)
X = torch.tensor(Xa, device=device)
Y = torch.tensor(Ya, device=device)
print(f"{X.shape=}")
print(f"{Y.shape=}")

In [ ]:
n1 = int(len(X) * 0.8)
n2 = int(len(X) * 0.9)
print("Split Global Dataset on lines:", n1, n2)
X_tr = X[:n1]
Y_tr = Y[:n1]
X_val = X[n1:n2]
Y_val = Y[n1:n2]
X_tst = X[n2:]
Y_tst = Y[n2:]
print(f"{X_tr.shape=}")
print(f"{X_val.shape=}")
print(f"{X_tst.shape=}")

In [ ]:
(5/3) * (hidden**-0.5) 

In [ ]:
#torch.manual_seed(42)
E_w = torch.randn(size=(voc_size, emb), device=device) * ((emb*att)**-0.5) * (5/3)
H0_w = torch.randn(size=(emb*att, hidden), device=device) * (hidden**-0.5) * (5/3)
H0_b = torch.randn(size=(hidden,), device=device) * 0.01
H1_w = torch.randn(size=(hidden, voc_size), device=device) * (voc_size)**-0.5
H1_b = torch.randn(size=(voc_size,), device=device) * 0.01
params = [E_w, H0_w, H0_b, H1_w, H1_b]
nparams = sum([t.numel() for t in params])
print(f"{nparams=}")
for t in params:
    t.requires_grad_()

In [ ]:
def forward(X):
    Ey = E_w[X].flatten(1, 2)
    H0_y = torch.tanh(Ey @ H0_w + H0_b)
    L = H0_y @ H1_w + H1_b
    return L

def calc_loss(L, Y):
    return F.cross_entropy(L, Y)

def backward(loss: torch.Tensor):
    for p in params:
        if p.grad is not None:
            p.grad.zero_()
    loss.backward()

def update_params(step):
    for p in params:
        p.data -= step * p.grad


In [ ]:
def get_batch(X0, Y0, n):
    rids = torch.randint(0, n1, (n,), device=device)
    return X0[rids], Y0[rids]

In [ ]:
LL=[]

batch = 32
X0, Y0 = get_batch(X_tr, Y_tr, batch)

In [ ]:
# Training Loop
L = forward(X0)
loss = calc_loss(L, Y0)
print("No Training. loss:", loss.item())

# initial training
for i in range(1):
    backward(loss)
    update_params(0.1)
    X0, Y0 = get_batch(X_tr, Y_tr, batch)
    L = forward(X0)
    loss = calc_loss(L, Y0)
print("After Initial training. loss: ", loss.item())


In [ ]:
print("dead neurons:", torch.where(torch.abs(H0_w.grad)<0.0001, 1,0).sum())
with torch.no_grad():
    X0, Y0 = X_val, Y_val
    Ey = E_w[X0].flatten(1, 2)
    H0_y = torch.tanh(Ey @ H0_w + H0_b)
    L = H0_y @ H1_w + H1_b
    gg = L.flatten().detach().cpu().numpy()
    print(gg.shape)
    plt.hist(gg, bins=50)

In [ ]:

N = 10000
steps=[0.1, 0.03, 0.01, 0.003, 0.0001]
#steps=[0.1]
#steps=[0.01]
#steps=[0.03]
#steps=[0.0001]
WIN = []
for step in steps:
    for i in range(N):
        backward(loss)
        update_params(step)
        X0, Y0 = get_batch(X_tr, Y_tr, batch)
        L = forward(X0)
        loss = calc_loss(L, Y0)
        WIN.append(loss.item())
        if i % 200 == 0:
          LL.append(np.mean(WIN))
          WIN = []
        
    print(f"Step {step} done. Loss: {LL[-1]}")

In [ ]:
# Validation / Test
with torch.no_grad():
    L = forward(X_val)
    loss = calc_loss(L, Y_val)
    print("Validation loss", loss)

    L = forward(X_tst)
    loss = calc_loss(L, Y_tst)
    print("Test loss", loss)

In [ ]:
# Plot loss
df = pd.DataFrame(LL, columns=['X'])
X_col=df['X']
MA_X_col = df['X'].rolling(window=20).mean()
plt.figure(figsize=(20, 5))
plt.plot(X_col, 'b-', label='loss')
plt.plot(MA_X_col, 'r-', label='MA20')
plt.grid(linestyle='--')
plt.legend(loc='upper center')
plt.show()
print("Tail of Moving Average column")
MA_X_col[-10:]

In [ ]:
s=encode("emm")

for i in range(100):
    x = torch.tensor([s[-att:]], device=device)
    L = forward(x)
    L = torch.softmax(L, dim=-1)
    ci = torch.multinomial(L, num_samples=1).item()
    #ci = int(torch.argmax(L).item())
    c = itos[ci]
    if ci == 0:
        break
    s += [ci]
print(decode(s))

In [ ]:
beg = []
batch = words[30:55]
for w in batch:
    if len(w) < att:
        w = "." * (len(w) - att) + w
    beg.append(encode(w[:att]))
x = torch.tensor(beg, device=device)
#print(x)
for i in range(7):
    L = forward(x[:,-att:])
    L = torch.softmax(L, dim=-1)
    y = torch.multinomial(L, num_samples=1)
    #y = torch.argmax(L, dim=1, keepdim=True)
    x = torch.cat([x,y],dim=-1)
for i, row in enumerate(x.detach().cpu().numpy()):
    print(decode(row, True), "   ", batch[i])


In [ ]:
def plot_emb():
    plt.figure(figsize=(8, 8))
    plt.scatter(E_w[:,0].detach().cpu().numpy(), E_w[:,1].detach().cpu().numpy(), s=200)
    for i in range(voc_size):
        plt.text(E_w[i,0].item(), E_w[i,1].item(), itos[i], ha="center", va="center", color="white")

In [ ]:
plot_emb()